In [1]:
import glob

In [2]:
import os

In [3]:
import cv2

In [4]:
import numpy as np

In [5]:
from glob import glob

In [6]:
import logging

In [8]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

In [9]:
from tensorflow.keras.applications import imagenet_utils

In [10]:
logging.basicConfig(level=logging.DEBUG)

In [11]:
print('hello')

hello


In [12]:
# Fonction de test pour vérifier l'importation des modèles
def test_model_import(model_name, model_class, weights='imagenet'):
    try:
        model = model_class(weights=weights)
        logging.info(f"{model_name} importé et chargé avec succès.")
        return model
    except Exception as e:
        logging.error(f"Erreur lors de l'importation de {model_name}: {e}")
        return None

In [13]:
# Tester l'importation de ResNet50V2
resnet50v2 = test_model_import("ResNet50V2", ResNet50V2)

INFO:root:ResNet50V2 importé et chargé avec succès.


In [14]:
# Mappage des identifiants de classes ImageNet à des noms de classes spécifiques
class_name_mapping = {
    'n01440764': 'tench',
    'n01443537': 'goldfish',
    'n01484850': 'great_white_shark',
    'n01491361': 'tiger_shark',
    'n01494475': 'hammerhead',
    'n01496331': 'electric_ray',
    'n01498041': 'stingray',
    'n01514668': 'cock',
    'n01514859': 'hen',
    'n01518878': 'ostrich',
    'n01530575': 'brambling',
    'n01531178': 'goldfinch',
    'n01532829': 'house_finch',
    'n01534433': 'junco',
    'n01537544': 'indigo_bunting',
    "n01558993": "robin",
    "n01560419": "bulbul",
    "n01580077": "jay",
    "n01582220": "magpie",
    "n01592084": "chickadee",
    "n01601694": "water_ouzel",
    "n01608432": "kite",
    "n01614925": "bald_eagle",
    "n01616318": "vulture",
    "n01622779": "great_grey_owl",
    "n01629819": "European_fire_salamander",
    "n01630670": "common_newt",
    "n01631663": "eft",
    "n01632458": "spotted_salamander",
    "n01632777": "axolotl",
    "n01641577": "bullfrog",
    "n01644373": "tree_frog",
    "n01644900": "tailed_frog",
    "n01664065": "loggerhead",
    "n01665541": "leatherback_turtle",
     'n07873807': 'pizza',
    'n07753592': 'banana',
    'n02504458': 'African_elephant',
    'n03291819': 'table',
    'n02979186': 'camera',
    'n03793489':'mouse',
    'n03196217':'digital_clock',
    'n04548280':'wall_clock',
    'n03197337':'digital_watch',
    'n04356056':'sunglasses',
    'n03085013':'computer_keyboard',
    'n03180011':'desktop_computer',
    'n04355933':'sunglass',
    'n02948072':'candle',
    'n03594734':'jean',
    'n02999410':'chain',
    'n04120489':'running_shoe',
    'n04370456':'sweatshirt',
    'n03617480':'kimono',
    'n03916031':'perfume',
    'n04069434':'reflex_camera',
    'n03976467':'Polaroid_camera',
    'n04350905':'suit',
    'n03047690':'clog',
    'n03595614':'jersey',
    'n04264628':'space_bar'
}


In [15]:
# Crée un dossier pour les images traitées
processed_image_folder = 'processed_images'
os.makedirs(processed_image_folder, exist_ok=True)

In [16]:
def predict_image_class(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Impossible de charger l'image à partir du chemin: {img_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))


# Sauvegarde de l'image traitée
    processed_image_path = os.path.join(processed_image_folder, os.path.basename(img_path))
    cv2.imwrite(processed_image_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    logging.info(f"Image traitée sauvegardée dans : {processed_image_path}")

    resized_img = np.expand_dims(img, axis=0)
    final_image = preprocess_input(resized_img)

    predictions = resnet50v2.predict(final_image)
    results = imagenet_utils.decode_predictions(predictions)

    # Affichage des résultats de la prédiction
    predicted_class, _, _ = results[0][0]
    logging.debug(f"Classe prédite par le modèle (avant mappage): {predicted_class}")

    # Mappage de la classe prédite à une classe plus spécifique (si disponible)
    mapped_class = class_name_mapping.get(predicted_class, 'Classe inconnue')
    logging.debug(f"Classe après mappage: {mapped_class}")

    predicted_class_folder = 'dataset/' + mapped_class.lower()

    if not os.path.exists(predicted_class_folder):
        raise ValueError(f"Le dossier pour la classe {mapped_class} n'existe pas dans le chemin {predicted_class_folder}")
    
    class_images = glob(predicted_class_folder + '/*.jpg')
    class_images = [os.path.basename(image_path) for image_path in class_images]

    # Affichage des images similaires trouvées
    logging.debug(f"Images similaires trouvées: {class_images}")

    return predicted_class_folder, class_images


In [17]:
# Test de la fonction avec une image existante
if __name__ == '__main__':
    test_image_path = 'front/public/dataset/candle/candle1.jpg'  # Chemin de l'image à tester
    try:
        if resnet50v2:  # Vérifie si le modèle a été correctement chargé
            predicted_class_folder, class_images = predict_image_class(test_image_path)
            print(f"Classe prédite : {predicted_class_folder}")
            print(f"Images similaires : {class_images}")
        else:
            logging.error("Le modèle ResNet50V2 n'a pas pu être chargé. Impossible de continuer.")
    except Exception as e:
        logging.error(f"Erreur lors de la prédiction : {str(e)}")


INFO:root:Image traitée sauvegardée dans : processed_images\candle1.jpg
DEBUG:root:Classe prédite par le modèle (avant mappage): n02948072
DEBUG:root:Classe après mappage: candle
ERROR:root:Erreur lors de la prédiction : Le dossier pour la classe candle n'existe pas dans le chemin dataset/candle
